# NOTE :
### 1. Models trained/evaluated in final_ptge.ipynb havce been saved and then loaded. Pretrained models have been used here, hence please change the path to the pretrained models accordingly. We can also merge this code with the final_ptge.ipynb if we dont want to use pretrained models.
### 2. One can download my pretrained models as well. Details are provided in README,md

### 3. Due to limited compute and storage resources, we have considered to process and work with only 3 subjects [p00,p01,p02]. For the leave-out strategy we have used only p00
### 4. Make sure you have already executed "python3 prepare_and_process_data.py". This will created "processed" folder need to run this notebook


In [1]:
!pip install tensorflow==2.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [3]:
!pip install tf-models-official==2.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 31.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=20bcd84883132f30f9b9f13b6c401c14f548109903d4440b95910b818accaf09
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
import tensorflow as tf
tf.version.VERSION

2024-07-12 16:06:28.761791: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 16:06:28.761851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 16:06:28.763604: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'2.15.1'

In [5]:
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
import random
import numpy as np
import os, glob
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.models import Model
from PIL import Image

gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
print(gpus)
import matplotlib.pyplot as plt

# Ensure the same seed for reproducibility
random.seed(12)
np.random.seed(12)
tf.random.set_seed(12)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
# If you have any custom objects (e.g., custom metrics or layers), define/import them here.
class AngularError(tf.keras.metrics.Metric):
    def __init__(self, name='mean_angular_error', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_error = self.add_weight(name='total_error', initializer='zeros')
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.math.l2_normalize(y_true, axis=-1)
        y_pred = tf.math.l2_normalize(y_pred, axis=-1)
        dot_product = tf.reduce_sum(y_true * y_pred, axis=-1)
        dot_product = tf.clip_by_value(dot_product, -1.0, 1.0)
        angular_error = tf.acos(dot_product)
        angular_error = angular_error * 57.296
        self.total_error.assign_add(tf.reduce_sum(angular_error))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_error / self.num_samples

    def reset_state(self):
        self.total_error.assign(0.0)
        self.num_samples.assign(0.0)


# Comparing PTGE and SPAZE model's robustness in handling variations in calibration parameters.

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [9]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image


# Define custom Huber loss function
def custom_huber_loss(y_true, y_pred, delta=1.5):
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= delta

    small_error_loss = tf.square(error) / 2
    big_error_loss = delta * (tf.abs(error) - delta / 2)

    return tf.where(is_small_error, small_error_loss, big_error_loss)


# Calculate gaze loss as the average of Huber losses
def gaze_loss(y_true, y_pred, delta=1.5):
    huber_losses = custom_huber_loss(y_true, y_pred, delta)
    return tf.reduce_mean(huber_losses)

# Function to preprocess the images
def preprocess_image(image, target_size):
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Define the GazeDataset class
class GazeDataset(tf.data.Dataset):
    def __new__(cls, subject_to_leave_out=None, batch_size=8, validation=False):
        def _generator():
            root_dir = 'processed_data'
            subjects = ['p00', 'p01', 'p02']
            transform_face = lambda img: preprocess_image(img, (224, 224))
            transform_eye = lambda img: preprocess_image(img, (112, 112))

            for subject in subjects:
                if validation:
                    if subject != subject_to_leave_out:
                        continue
                else:
                    if subject == subject_to_leave_out:
                        continue
                
                person_dir = os.path.join(root_dir, 'Image', subject)
                for image_name in os.listdir(os.path.join(person_dir, 'face')):
                    face_image_path = os.path.join(person_dir, 'face', image_name)
                    left_eye_image_path = os.path.join(person_dir, 'lefteye', image_name)
                    right_eye_image_path = os.path.join(person_dir, 'righteye', image_name)
                    rotation_matrix_path = os.path.join(person_dir, 'rotation_matrix', image_name.replace('.jpg', '.npy'))
                    rotation_matrix_flipped_path = os.path.join(person_dir, 'rotation_matrix_flipped', image_name.replace('.jpg', '.npy'))
                    gaze_2d_path = os.path.join(person_dir, '2d_gaze', image_name.replace('.jpg', '.npy'))
                    gaze_3d_path = os.path.join(person_dir, '3d_gaze', image_name.replace('.jpg', '.npy'))
                    gaze_3d_flipped_path = os.path.join(person_dir, '3d_gaze_flipped', image_name.replace('.jpg', '.npy'))
                    eye_coords_path = os.path.join(person_dir, 'eye_coords', image_name.replace('.jpg', '.npy'))

                    face_image = Image.open(face_image_path).convert('RGB')
                    left_eye_image = Image.open(left_eye_image_path).convert('RGB')
                    right_eye_image = Image.open(right_eye_image_path).convert('RGB')

                    face_image = transform_face(np.array(face_image))
                    left_eye_image = transform_eye(np.array(left_eye_image))
                    right_eye_image = transform_eye(np.array(right_eye_image))

                    rotation_matrix = np.load(rotation_matrix_path).astype(np.float32)
                    rotation_matrix_flipped = np.load(rotation_matrix_flipped_path).astype(np.float32)
                    gaze_2d = np.load(gaze_2d_path).astype(np.float32)
                    gaze_3d = np.load(gaze_3d_path).astype(np.float32)
                    gaze_3d_flipped = np.load(gaze_3d_flipped_path).astype(np.float32)
                    eye_coords = np.load(eye_coords_path).astype(np.float32)

                    yield face_image, left_eye_image, right_eye_image, rotation_matrix, rotation_matrix_flipped, gaze_2d, gaze_3d, gaze_3d_flipped, eye_coords, subject

        return tf.data.Dataset.from_generator(
            _generator,
            output_signature=(
                tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(112, 112, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(112, 112, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(3, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(3, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(2,), dtype=tf.float32),
                tf.TensorSpec(shape=(3,), dtype=tf.float32),
                tf.TensorSpec(shape=(3,), dtype=tf.float32),
                tf.TensorSpec(shape=(6,), dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.string)
            )
        ).batch(batch_size)

# Testing the dataset
val_dataloader = GazeDataset(subject_to_leave_out='p00', batch_size=8)

In [10]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Define the evaluation function with calibration variations
def evaluate_model_with_calibration_variations(model, dataloader, model_type='ptge', variation_range=(-0.1, 0.1)):
    total_loss = 0.0
    num_batches = 0
    results = []

    for i, (face_image, left_eye_image, right_eye_image, rotation_matrix, rotation_matrix_flipped, gaze_2d, gaze_3d, gaze_3d_flipped, eye_coords, subject_id) in enumerate(dataloader):
        # Apply variations to calibration parameters (rotation_matrix)
        variation = np.random.uniform(variation_range[0], variation_range[1], rotation_matrix.shape)
        varied_rotation_matrix = rotation_matrix + variation

        subject_indices = [int(s.decode().split('p')[1]) for s in subject_id.numpy()]

        input_dict = {
            'eye_coords': tf.convert_to_tensor(eye_coords, dtype=tf.float32),
            'face': tf.convert_to_tensor(face_image, dtype=tf.float32),
            'flipped_face': tf.convert_to_tensor(tf.image.flip_left_right(face_image), dtype=tf.float32),
            'id': tf.convert_to_tensor(subject_indices, dtype=tf.int32),
            'lefteye': tf.convert_to_tensor(left_eye_image, dtype=tf.float32),
            'righteye': tf.convert_to_tensor(right_eye_image, dtype=tf.float32),
            'rotation_matrix': tf.convert_to_tensor(varied_rotation_matrix, dtype=tf.float32)
        }

        calibration_input_dict = input_dict.copy()
        calibration_input_dict['rotation_matrix_flipped'] = tf.convert_to_tensor(rotation_matrix_flipped, dtype=tf.float32)

        #print(f"Batch {i} - Input Shapes and Types:")
        #for key, value in input_dict.items():
        #    print(f"{key}: shape={value.shape}, dtype={value.dtype}")

        try:
            if model_type == 'ptge':
                # First, get the initial gaze estimation from the Gaze Model
                initial_gaze_estimation = model['gaze_model'](input_dict)
                #print(f"Initial gaze estimation shape: {initial_gaze_estimation.shape}")

                # Now, use the Calibration Model to refine the gaze estimation
                calibration_input_dict['gaze'] = initial_gaze_estimation
                calibration_input_dict['gaze_flipped'] = initial_gaze_estimation  # No flipping, use as is

                refined_gaze_estimation = model['calibration_model'](calibration_input_dict)
                #print(f"Refined gaze estimation shape before reshape: {refined_gaze_estimation.shape}")
                #print(f"Refined gaze estimation content: {refined_gaze_estimation.numpy()}")

                # Ensure the refined_gaze_estimation shape matches the gaze_3d shape
                refined_gaze_estimation = refined_gaze_estimation[:, :3]  # Only take the first 3 columns
                #print(f"Reshaping refined_gaze_estimation from {refined_gaze_estimation.shape} to {gaze_3d.shape}")

                # Calculate loss
                loss = gaze_loss(gaze_3d, refined_gaze_estimation)
                results.append((gaze_3d.numpy(), refined_gaze_estimation.numpy()))

            elif model_type == 'spaze':
                # Get the gaze estimation from the SPAZE Model
                spaze_gaze_estimation = model['spaze_model'](face_image, training=False)
                #print(f"SPAZE gaze estimation shape: {spaze_gaze_estimation.shape}")

                # Calculate loss
                loss = gaze_loss(gaze_2d, spaze_gaze_estimation)
                results.append((gaze_2d.numpy(), spaze_gaze_estimation.numpy()))

            total_loss += loss.numpy()
            num_batches += 1

        except Exception as e:
            print(f"Error during gaze model prediction: {str(e)}")
            continue

    average_loss = total_loss / num_batches if num_batches > 0 else float('inf')
    return average_loss, results

# Define the function to evaluate both models and save results
def evaluate_and_save_results(subjects, variation_ranges):
    # Initialize the results dictionary
    results = {
        'variation_range': [],
        'ptge_loss': [],
        'spaze_loss': [],
    }

    for subject in subjects:
        print(f"Evaluating for subject: {subject}")
        val_dataloader = GazeDataset(subject_to_leave_out=subject, batch_size=8)

        for variation_range in variation_ranges:
            print(f"Evaluating with variation range: {variation_range}")

            # Evaluate PTGE model with calibration variations
            ptge_loss, _ = evaluate_model_with_calibration_variations(
                {'gaze_model': gaze_model, 'calibration_model': calibration_model}, 
                val_dataloader, 
                model_type='ptge', 
                variation_range=variation_range
            )
            # Print PTGE loss
            print(f"PTGE Loss for variation range {variation_range}: {ptge_loss}")
            
            # Evaluate SPAZE model with calibration variations
            spaze_loss, _ = evaluate_model_with_calibration_variations(
                {'spaze_model': spaze_model}, 
                val_dataloader, 
                model_type='spaze', 
                variation_range=variation_range
            )
            # Print SPAZE loss
            print(f"SPAZE Loss for variation range {variation_range}: {spaze_loss}")
            
            results['variation_range'].append(variation_range)
            results['ptge_loss'].append(ptge_loss)
            results['spaze_loss'].append(spaze_loss)

    # Convert the results to a DataFrame and save
    df_results = pd.DataFrame(results)
    df_results.to_csv('ptge_spaze_robustness_results.csv', index=False)
    print("Results saved to 'ptge_spaze_robustness_results.csv'.")

# Initialize models
gaze_model = tf.keras.models.load_model('/kaggle/input/gazing-all/gaze_model_with_leaveout_p00/kaggle/working/gaze_model_with_leaveout_p00', compile=False)
calibration_model = tf.keras.models.load_model('/kaggle/input/gazing-all/calibration_model_with_leaveout_p00/kaggle/working/calibration_model_with_leaveout_p00', compile=False)

# Assume spaze_model is defined elsewhere in your code
spaze_model = tf.keras.models.load_model('/kaggle/input/gazing-all/spaze_model_with_leaveout_p00/kaggle/working/spaze_model_with_leaveout_p00', compile=False)

# Define subjects and variation ranges
subjects = ['p00']
variation_ranges = [(-0.1, 0.1), (-0.2, 0.2), (-0.3, 0.3)]

# Evaluate and save results
evaluate_and_save_results(subjects, variation_ranges)


Evaluating for subject: p00
Evaluating with variation range: (-0.1, 0.1)
PTGE Loss for variation range (-0.1, 0.1): 0.16863020888964336
SPAZE Loss for variation range (-0.1, 0.1): 107.64021548461913
Evaluating with variation range: (-0.2, 0.2)
PTGE Loss for variation range (-0.2, 0.2): 0.16863009786605834
SPAZE Loss for variation range (-0.2, 0.2): 107.64021548461913
Evaluating with variation range: (-0.3, 0.3)
PTGE Loss for variation range (-0.3, 0.3): 0.1686294214129448
SPAZE Loss for variation range (-0.3, 0.3): 107.64021548461913
Results saved to 'ptge_spaze_robustness_results.csv'.
